<a href="https://colab.research.google.com/github/Amplil/pytorch/blob/master/7_4_vectorize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.4 word2vec、fastTextを用いた日本語単語のベクトル表現の実装

- 本ファイルでは、日本語の単語をword2vecもしくはfastTextを使用してベクトル化する手法を解説します。

※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.4 学習目標

1.	学習済みの日本語word2vecモデルで単語をベクトル表現に変換する実装ができるようになる
2.	学習済みの日本語fastText モデルで単語をベクトル表現に変換する実装ができるようになる


# 事前準備
書籍の指示に従い、本章で使用するデータを用意します

pip install gensim


# 1. 文書を読み込んで、分かち書き、データセット作成まで（8.2と同じです）

前処理と分かち書きをし、最後にデータセットを作成する部分を実装します


In [ ]:
#!cd /content

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

In [3]:
!apt install mecab
!apt install libmecab-dev
!apt install mecab-ipadic-utf8
!git clone https://github.com/neologd/mecab-ipadic-neologd.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab2 mecab mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 5 newly installed, 0 to remove and 31 not upgraded.
Need to get 16.5 MB of archives.
After this operation, 219 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic-utf8 all 7.0-20130310-4 [16.2 MB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic all 7.0-20130310-4 [2,212 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab amd64 0.996-5 [132 kB]
Fetched 16.5 MB in 2

In [4]:
cd /content/mecab-ipadic-neologd

/content/mecab-ipadic-neologd


In [5]:
!apt install file
!./bin/install-mecab-ipadic-neologd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 31 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Fetched 275 kB in 1s (475 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 161161 files and directories currently installed.)
Preparing to unpack .../libmagic-mgc_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic-mgc (1:5.32

In [14]:
cd "/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer"

/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer


In [ ]:
#pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" unidic-lite

In [ ]:
#pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" gensim

In [15]:
# 単語分割にはMecab＋NEologdを使用
import MeCab

m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret



# 前処理として正規化をする関数を定義
import re

def preprocessing_text(text):
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    return text


# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Mecabの単語分割

    return ret


In [16]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)


# フォルダ「data」から各tsvファイルを読み込みます
train_ds, val_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 2. 単語のベクトル化

## 2.1 word2vec

単語をベクトル表現に変換します。

TorchTextには日本語の学習済みデータがないわけではないですが、精度が微妙なので

東北大学 乾・岡崎研究室で公開されているWord2Vecの学習済みのベクトルを使用します。



In [ ]:
# 以下から、日本語のfasttextの学習済みベクトルをダウンロードします

# 東北大学 乾・岡崎研究室：日本語 Wikipedia エンティティベクトル

# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/
# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2

In [ ]:
#!wget http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2

--2021-03-24 09:58:28--  http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1373795477 (1.3G) [application/x-bzip2]
Saving to: ‘20170201.tar.bz2’

20170201.tar.bz2    100%[===================>]   1.28G  10.9MB/s    in 2m 3s   

2021-03-24 10:00:32 (10.6 MB/s) - ‘20170201.tar.bz2’ saved [1373795477/1373795477]



In [4]:
cd "/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer"

/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer


In [11]:
# そのままではtorchtextで読み込めないので、gensimライブラリを使用して、
# Word2Vecのformatで保存し直します

# 事前インストール
# pip install gensim

from gensim.models import KeyedVectors


# 一度gensimライブラリで読み込んで、word2vecのformatで保存する
model = KeyedVectors.load_word2vec_format(
    './../data/entity_vector/entity_vector.model.bin', binary=True)

# 保存（時間がかかります、10分弱）
model.wv.save_word2vec_format('./../data/japanese_word2vec_vectors.vec')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [9]:
import os
import urllib.request
import zipfile
import tarfile
# フォルダ「data」が存在しない場合は作成する
data_dir = "/content/data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [10]:
cp ./../data/japanese_word2vec_vectors.vec /content/data

In [17]:
cd /content

/content


In [12]:
# torchtextで単語ベクトルとして読み込みます
from torchtext.vocab import Vectors

japanese_word2vec_vectors = Vectors(
    name='./data/japanese_word2vec_vectors.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_word2vec_vectors.dim)
print("単語数：", len(japanese_word2vec_vectors.itos))


100%|█████████▉| 1014810/1015474 [01:26<00:00, 11950.13it/s]

1単語を表現する次元数： 200
単語数： 1015474


In [18]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 49個の単語が200次元のベクトルで表現されている
TEXT.vocab.vectors

torch.Size([49, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 2.6023, -2.6357, -2.5822,  ...,  0.6953, -1.4977,  1.4752],
        ...,
        [-2.8353,  2.5609, -0.5348,  ...,  0.4602,  1.4669, -2.1255],
        [-1.5885,  0.1614, -0.6029,  ..., -1.7545, -1.2462,  2.3034],
        [-0.0448, -0.1304,  0.0329,  ...,  0.0825, -0.1386,  0.0417]])

In [19]:
# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f6437e7b110>>,
            {'0': 18,
             '<pad>': 1,
             '<unk>': 0,
             '、': 7,
             '。': 3,
             'い': 19,
             'いる': 11,
             'か': 12,
             'から': 20,
             'が': 8,
             'し': 9,
             'する': 21,
             'その': 22,
             'た': 23,
             'て': 13,
             'で': 24,
             'です': 25,
             'と': 2,
             'な': 4,
             'に': 26,
             'に対して': 27,
             'の': 5,
             'は': 28,
             'まし': 29,
             'ます': 14,
             'を': 10,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '分類': 15,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
             '女王': 39,
             '好き': 40,
             '姫': 41,
     

In [20]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3840)
王 tensor(0.3669)
王子 tensor(0.5489)
機械学習 tensor(-0.1404)


姫 - 女性 + 男性　を計算すると狙った通り、王子がもっとも近い結果になりました

## 2.2 fastText

word2vecより進歩したベクトル化手法であるfastTextによる単語のベクトル表現を使用します。

日本語の学習モデルを以下の記事にて公開してくださっているので、使用させていただきます。



In [ ]:
# Qiita：いますぐ使える単語埋め込みベクトルのリスト
# https://qiita.com/Hironsan/items/8f7d35f0a36e0f99752c

# Download Word Vectors
# https://drive.google.com/open?id=0ByFQ96A4DgSPNFdleG1GaHcxQzA

In [21]:
cd "/content/drive/My Drive/Colab Notebooks/pytorch_advanced"

/content/drive/My Drive/Colab Notebooks/pytorch_advanced


In [22]:
# torchtextで単語ベクトルとして読み込みます
# word2vecとは異なり、すぐに読み込めます

from torchtext.vocab import Vectors

japanese_fasttext_vectors = Vectors(name='./data/vector_neologd/model.vec')

                                    
# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", japanese_fasttext_vectors.dim)
print("単語数：", len(japanese_fasttext_vectors.itos))


1単語を表現する次元数： 300
単語数： 351122


In [23]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=japanese_fasttext_vectors, min_freq=1)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 52個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


torch.Size([49, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f63fe988350>>,
            {'0': 18,
             '<pad>': 1,
             '<unk>': 0,
             '、': 7,
             '。': 3,
             'い': 19,
             'いる': 11,
             'か': 12,
             'から': 20,
             'が': 8,
             'し': 9,
             'する': 21,
             'その': 22,
             'た': 23,
             'て': 13,
             'で': 24,
             'です': 25,
             'と': 2,
             'な': 4,
             'に': 26,
             'に対して': 27,
             'の': 5,
             'は': 28,
             'まし': 29,
             'ます': 14,
             'を': 10,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '分類': 15,
             '取り組み': 36,
             '商品': 37,
             '女性': 38,
             '女王': 39,
             '好き': 40,
             '姫': 41,
     

In [24]:
# 姫 - 女性 + 男性 のベクトルがどれと似ているのか確認してみます
import torch.nn.functional as F

# 姫 - 女性 + 男性
tensor_calc = TEXT.vocab.vectors[41] - \
    TEXT.vocab.vectors[38] + TEXT.vocab.vectors[46]

# コサイン類似度を計算
# dim=0 は0次元目で計算してくださいという指定
print("女王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[39], dim=0))
print("王", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[44], dim=0))
print("王子", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[45], dim=0))
print("機械学習", F.cosine_similarity(tensor_calc, TEXT.vocab.vectors[43], dim=0))


女王 tensor(0.3650)
王 tensor(0.3461)
王子 tensor(0.5531)
機械学習 tensor(0.0952)


姫 - 女性 + 男性　を計算すると狙った通り、王子がもっとも近い結果になりました

以上